In [226]:
import numpy as np

import torch
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.models import resnet34
from torch.utils.data import DataLoader

from sklearn.metrics import confusion_matrix, f1_score
import random
import math
from tqdm import tqdm

from numpy.ma.core import ceil
from scipy.spatial import distance #distance calculation
from sklearn.preprocessing import MinMaxScaler #normalisation
from sklearn.metrics import accuracy_score #scoring
import matplotlib.pyplot as plt
from matplotlib import colors

In [227]:
transform = transforms.Compose([
    transforms.Resize((128, 128)), # Resize to 224x224 (height x width)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

In [241]:
# loading the train data
batch_size = 1

train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
train_dataloader = DataLoader(train_data, batch_size=batch_size,shuffle=True )

#loading the test data
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)
test_dataloader = DataLoader(test_data,batch_size=batch_size, shuffle=True)


Files already downloaded and verified
Files already downloaded and verified


In [242]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Feature Extractor

In [243]:
feature_extractor = resnet34(pretrained=True)
num_features = feature_extractor.fc.in_features

for param in feature_extractor.parameters():
    param.requires_grad = False

feature_extractor.fc = nn.Identity()
feature_extractor.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Helper Functions

In [244]:

# Manhattan distance
def manhattan_distance(x, y):
  return distance.cityblock(x,y)

# Euclidean distance
def euclidean_distance(x, y):
    return torch.sqrt(torch.sum((x - y) ** 2, dim=-1))


# Best Matching Unit search
def bmu_search(data, som, num_rows, num_cols):
  winner = [0,0]
  # som = som.to(device)
  # data = data.to(device)
  shortest_distance = 10e6 
  for row in range(num_rows):
    for col in range(num_cols):
      if som[row][col] != None:
        
        distance = euclidean_distance(som[row][col], data)
        if distance < shortest_distance: 
          shortest_distance = distance
          winner = [row,col]
  return winner

#guassian
def distance_func(x):
  sig = 2 
  return np.exp(-np.power(x , 2.) / (2 * np.power(sig, 2.)))
  

In [245]:
### Optimizer
class FactorScheduler:
    def __init__(self, factor=1, stop_factor_lr=1e-7, base_lr=0.001):
        self.factor = factor
        self.stop_factor_lr = stop_factor_lr
        self.base_lr = base_lr

    def __call__(self, num_update):
        #self.base_lr = max(self.stop_factor_lr, self.base_lr * self.factor)
        return self.base_lr

class CosineScheduler:
    def __init__(self, max_update, base_lr=0.01, final_lr=0,
               warmup_steps=0, warmup_begin_lr=0):
        self.base_lr_orig = base_lr
        self.max_update = max_update
        self.final_lr = final_lr
        self.warmup_steps = warmup_steps
        self.warmup_begin_lr = warmup_begin_lr
        self.max_steps = self.max_update - self.warmup_steps

    def get_warmup_lr(self, epoch):
        increase = (self.base_lr_orig - self.warmup_begin_lr) \
                       * float(epoch) / float(self.warmup_steps)
        return self.warmup_begin_lr + increase

    def __call__(self, epoch):
        if epoch < self.warmup_steps:
            return self.get_warmup_lr(epoch)
        if epoch <= self.max_update:
            self.base_lr = self.final_lr + (
                self.base_lr_orig - self.final_lr) * (1 + math.cos(
                math.pi * (epoch - self.warmup_steps) / self.max_steps)) / 2
        return self.base_lr
        #return self.base_lr_orig

# Learning rate and neighbourhood range calculation
def neighborhood_optimizer(step, max_steps, max_m_distance):
  coefficient = 1.0 - (np.float64(step)/max_steps)
  # neighbourhood_range = ceil(coefficient * max_m_distance)
  neighbourhood_range = max_m_distance
  return neighbourhood_range

### Feature Extracting Train Data

In [246]:
# y_data_list = []
# data_list = []
# for x_train, y_train in train_dataloader:
#   x_train, y_train = x_train.to(device), y_train.to(device)

#   features = feature_extractor(x_train)
#   features = minmax_scaler(features.cpu().numpy())
#   features = torch.from_numpy(features)
#   data_list.append(features)

#   y_data_list.append(y_train)

# print(len(y_data_list))
# print(len(data_list))

### Feature Extracting Test Data

In [247]:
# y_test_list = []
# data_test_list = []
# for x_test, y_test in test_dataloader:
#   x_test, y_test = x_test.to(device), y_test.to(device)

#   features = feature_extractor(x_test)
#   features = minmax_scaler(features.cpu().numpy())
#   features = torch.from_numpy(features)
#   data_test_list.append(features)

#   y_test_list.append(y_test)

# print(len(y_test_list))
# print(len(data_test_list))

## Hyperparameters

In [248]:
num_rows = 4
num_cols = 3
max_neighborhood_range = 1
max_learning_rate = 0.01
max_steps = 20
is_2d_10_neuron = True


In [249]:
batch_size= 1
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
som_init_dataloader = DataLoader(train_data, batch_size=batch_size,shuffle=True )
num_features = 512

def initial_som():

  unique_labels = list(set(train_data.targets))
  random.shuffle(unique_labels)
  selected_samples = []

  # Iterate over the train_dataloader until we have selected enough samples
  for x_train, y_train in som_init_dataloader:
      x_train, y_train = x_train.to(device), y_train.to(device)
      feature = feature_extractor(x_train).cpu().numpy()
      

      # Check if the label is one of the unique labels and has not been selected already
      if y_train[0] not in [sample[1] for sample in selected_samples]:
        selected_samples.append(feature[0])

      # Break the outer loop if we have collected enough samples
      if len(selected_samples) == num_rows * num_cols:
          break



  # Initialize SOM with random values
  np.random.seed(40)
  som = np.random.random_sample(size=(num_rows, num_cols, num_features)) # map construction

  i =0
  for row in range(num_rows):
    for col in range(num_cols):
      if not is_2d_10_neuron:
        som[row][col] = selected_samples[i]
        i+=1
      else:
        if (row == 3 and col == 0) or (row == 3 and col == 2):
          som[row][col] = 10e9
        else:
          som[row][col] = selected_samples[i]
          i+=1
        som[3][0] = 10e9
        som[3][2] = 10e9
  return som


som = initial_som()
som = torch.from_numpy(som).to(device)
if is_2d_10_neuron:
  pass
  


# print(som.shape)
# for row in range(num_rows):
#     for col in range(num_cols):
#       if som[row][col] == None:
#         print(f'{row}, {col} nan')
#       else:
#         print(f'{row}, {col}')
# print(som[3][0])


Files already downloaded and verified


## Initializing Self Organising Map

In [250]:
# num_features = 512 # numnber of dimensions in the input data

# if is_2d_10_neuron:
#   np.random.seed(40)
#   som = np.random.random_sample(size=(num_rows, num_cols, num_features)) # map construction
#   som[3][0] = None
#   som[3][2] = None
#   som = torch.from_numpy(som).to(device)

# else:
#   np.random.seed(40)
#   som = np.random.random_sample(size=(num_rows, num_cols, num_features)) # map construction
#   som = torch.from_numpy(som).to(device)

# print(som.shape)

In [ ]:
epochs = 20



scheduler = CosineScheduler(max_update=epochs, base_lr=max_learning_rate, final_lr=0.0001)



for epoch in range(epochs):
    map = np.empty(shape=(num_rows, num_cols), dtype=object)
    for row in range(num_rows):
      for col in range(num_cols):
        map[row][col] = []
    final_features = torch.zeros(0,dtype=torch.long, device=device)
    final_y = torch.zeros(0,dtype=torch.long, device=device)
    bmus = []
    for x_train, y_train in tqdm(train_dataloader, desc=f"Epoch {epoch+1}", colour="blue"):
        x_train, y_train = x_train.to(device), y_train.to(device)
        features = feature_extractor(x_train)
        # final_features = features
        # final_y = y_train
        label_data = y_train.cpu().numpy()
        
        neighbourhood_range = neighborhood_optimizer(epoch, epochs, max_neighborhood_range)
        learning_rate=scheduler(epoch)
        
        # if epoch == epochs-1:
        final_features = (torch.cat([final_features] + [torch.tensor(f).view(1, -1) for f in features])).detach().clone()
        final_y = torch.cat([final_y,y_train.view(-1)])

        # start training iterations
        for i in range(features.shape[0]):
          bmu = bmu_search(features[i], som, num_rows, num_cols)
          map[bmu[0]][bmu[1]].append(label_data[i])
          bmus.append(bmu)
          for row in range(num_rows):
            for col in range(num_cols):
              if is_2d_10_neuron and (row == 3 and col ==0 or row ==3 and col ==2):
                pass
              else:
                dist = manhattan_distance([row, col], bmu)
                if dist <= neighbourhood_range:
                  som[row][col] += learning_rate * distance_func(dist) * (features[i].to(device) - som[row][col].to(device)) #update neighbour's weight

    label_map = np.zeros(shape=(num_rows, num_cols),dtype=np.int64)
    label_dispersion = np.zeros(shape=(num_rows, num_cols), dtype=np.float64)
    for row in range(num_rows):
      for col in range(num_cols):
        label_list = map[row][col]
        if len(label_list)==0:
          label = -3
          dispersion = 0.0
        else:
          label = max(label_list, key=label_list.count)
          count_label = label_list.count(label)
          count_all_labels = len(label_list)
          dispersion = count_label / count_all_labels

        label_map[row][col] = label
        label_dispersion[row][col] = dispersion 
    labels_from_bmu = np.zeros(shape=len(bmus),dtype=np.int64)
    for i, bmu in enumerate(bmus):
      labels_from_bmu[i] = label_map[bmu[0]][bmu[1]]
    acc = accuracy_score(final_y.cpu().numpy(), labels_from_bmu)
    f1 = f1_score(final_y.cpu().numpy(), labels_from_bmu, average='weighted')
    print("Accuracy: ", acc)
    print("F1 score:", f1)
    


Epoch 1:   0%|          | 0/50000 [00:00<?, ?it/s]<ipython-input-251-39060b991045>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  final_features = (torch.cat([final_features] + [torch.tensor(f).view(1, -1) for f in features])).detach().clone()
Epoch 1:  44%|████▎     | 21780/50000 [03:52<05:15, 89.38it/s]

In [ ]:
print(final_features.shape)
print(final_y.shape)

## Collecting Labels

In [ ]:
# map = np.empty(shape=(num_rows, num_cols), dtype=object)
# # final_features_np = final_features.cpu().numpy()
# # final_y_np = final_y.cpu().numpy()
# # print(final_features_np.shape[0])
# for row in range(num_rows):
#   for col in range(num_cols):
#     if som[row][col] != None:
#       map[row][col] = [] # empty list to store the label

# # for i, features in enumerate(final_features_np):

# label_data = final_y.cpu().numpy()

# for t in range(final_features.shape[0]):
  
#   bmu = bmu_search(final_features[t].to(device), som.to(device), num_rows, num_cols)
#   # print(bmu)
#   map[bmu[0]][bmu[1]].append(label_data[t]) # label of winning neuron

## Construct Label Map

In [ ]:
# label_map = np.zeros(shape=(num_rows, num_cols),dtype=np.int64)

# label_dispersion = np.zeros(shape=(num_rows, num_cols), dtype=np.float64)
# for row in range(num_rows):
#   for col in range(num_cols):
#     if som[row][col] != None:
#       label_list = map[row][col]
#       if len(label_list)==0:
#         label = -3
#         dispersion = 0.0
#       else:
#         label = max(label_list, key=label_list.count)
#         count_label = label_list.count(label)
#         count_all_labels = len(label_list)
#         dispersion = count_label / count_all_labels

#       label_map[row][col] = label
#       label_dispersion[row][col] = dispersion


In [ ]:
print(som[3][0])

In [ ]:
n=0
total = 0
for row in range(num_rows):
  for col in range(num_cols):
    total +=label_dispersion[row][col]
    n += 1

avg_dispersion = total/n

## Feature Map

In [ ]:
title = ('Feature Map')
fig, ax = plt.subplots(figsize=(10, 6))
plt.imshow(label_map, cmap='Blues')
ax.set_xticks(np.arange(num_cols))
ax.set_yticks(np.arange(num_rows))
ax.set_xticklabels(np.arange(1, num_cols+1))
ax.set_yticklabels(np.arange(1, num_rows+1))
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

for i in range(num_rows):
  for j in range(num_cols):
    text = ax.text(j, i, '{:.2f}'.format(label_map[i][j]),
                   ha="center", va="center", color="black")

plt.colorbar()
plt.title(title)
plt.show()

### dispersion map

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
im = ax.imshow(label_dispersion, cmap='Blues')
ax.set_xticks(np.arange(num_cols))
ax.set_yticks(np.arange(num_rows))
ax.set_xticklabels(np.arange(1, num_cols+1))
ax.set_yticklabels(np.arange(1, num_rows+1))
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

for i in range(num_rows):
  for j in range(num_cols):
    text = ax.text(j, i, '{:.2f}'.format(label_dispersion[i][j]),
                   ha="center", va="center", color="black")

plt.title('Label Dispersion Map')
plt.colorbar(im)
plt.show()

### Scatterplot of dispersion for each cluster

In [ ]:
# for i in range(num_rows):
#   for j in range(num_cols):
    
#     y = map[i][j]
#     # x = range(len(y))
#     print(label_map[i][j])
#     colors = ['green' if k == label_map[i][j] else 'blue' for k in y]
#     plt.hist(y, bins=11, c=colors)
#     # plt.scatter(x, y, c=colors)
#     # plt.xlabel('Index')
#     # plt.ylabel('Y values')
#     plt.show()

## Test Data
using the trained som, search the winning node of corresponding to the test data 

In [ ]:
sum_acc = 0
sum_f1 = 0
n = 0
for x_test, y_test in test_dataloader:
  x_test, y_test = x_test.to(device), y_test.to(device)
  features = feature_extractor(x_test)

  winner_labels = []

  for t in range(features.shape[0]):
    bmu = bmu_search(features[t], som, num_rows, num_cols)
    row = bmu[0]
    col = bmu[1]
    predicted = label_map[row][col]
    winner_labels.append(predicted)
  acc = accuracy_score(y_test.cpu().numpy(), winner_labels)
  f1 = f1_score(y_test.cpu().numpy(), winner_labels, average='weighted')
  sum_f1 += f1
  sum_acc += acc
  n += 1
  # print("Accuracy: ",acc)

print("Total Accuracy: ", sum_acc /n)
print("Average F1 score:", sum_f1 / n)
print("Average Dispersion :", avg_dispersion)